### Initialize a GPUTransformer to process the pixels' catalog table using GPU resources.

In [0]:
%run ./MONAILabel-ModelServing-init

In [0]:
from dbx.pixels.modelserving.bundles.servingendpoint import MonaiLabelBundlesTransformer

df = spark.table(table)

df_monai = MonaiLabelBundlesTransformer(table=table, destDir=os.environ["DEST_DIR"], endpointName=serving_endpoint_name, exportMetrics=True).transform(df)

display(df_monai.filter('series_uid = "1.2.156.14702.1.1000.16.1.2020031111365289000020001"'))

df_monai.repartition(8).write.saveAsTable(inference_results_table)

spark.table(inference_results_table).display()

# Test performance using noop
#df_monai.repartition(4).write.format("noop").mode("overwrite").save()

In [0]:
import torch
from dbx.pixels.modelserving.bundles.gpu import MonaiLabelBundlesGPUTransformer

gpuCount = int(spark.conf.get("spark.executor.resource.gpu.amount","0") or torch.cuda.device_count())
nWorkers = (int(spark.conf.get("spark.databricks.clusterUsageTags.clusterWorkers")) or 1)
tasksPerGpu = int(spark.conf.get("spark.task.resource.gpu.amount","1"))

df = spark.table(table)

df_monai = MonaiLabelBundlesGPUTransformer(inputCol="meta", 
                                 table=table, 
                                 destDir=os.environ["DEST_DIR"], 
                                 modelName=model_name,
                                 sqlWarehouseId=os.environ["DATABRICKS_WAREHOUSE_ID"], 
                                 labelPrompt=None, exportMetrics=True, exportOverlays=False, 
                                 secret=os.environ["DATABRICKS_TOKEN"], 
                                 host=os.environ["DATABRICKS_HOST"], 
                                 gpuCount=gpuCount, nWorkers=nWorkers, tasksPerGpu=tasksPerGpu).transform(df)

display(df_monai)

# Test performance using noop
#df_monai.write.format("noop").mode("overwrite").save()

In [0]:
%sql
-- Requires Databricks Runtime 15.2 and above or Serverless
-- Sample query to illustrate how to use the ai_query function to query model in serving endpoint 
with ct as (
  select distinct(meta:['0020000E'].Value[0]) as series_uid
  from IDENTIFIER(:table)
  where meta:['00080008'] like '%AXIAL%'
)

select series_uid, parse_json(ai_query(
  endpoint => :serving_endpoint_name,
  request => named_struct(
      'series_uid', series_uid,
      'params', named_struct(
                    'export_metrics', True,
                    'export_overlays', False
                )
  ),
  returnType => 'STRING'
)) as result from ct